In [1]:
import numpy as np
import os
import time
from vgg16 import VGG16
from keras.preprocessing import image
from keras_applications.imagenet_utils import preprocess_input
from imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import cv2

Using TensorFlow backend.


In [2]:
# Creating training_data_list

dir_path = "/home/parag/Transfer-Learning-in-keras---custom-data/data"  # current directory path

# you could use
#PATH = os.getcwd()
# data_path = PATH + '/data'
# data_dir_list = os.listdir(data_path)


CATEGORIES = ["cats", "dogs", "horses", "Humans"]    #4 categories

training_data = []          # empty list for training data for our model

img_size_setting = 224      # VGG require image size : 224*224*3           
               

def create_training_data():
    for category in CATEGORIES: 

        path = os.path.join(dir_path,category)  
        class_num = CATEGORIES.index(category)  # Setting labels:0= cats, 1= dogs, 2= horses, 3= Humans
        for img in tqdm(os.listdir(path)):  # iterate over each image per category
            try:
                img_array = cv2.imread(os.path.join(path,img))  #image read  
                resize_array = cv2.resize(img_array, (img_size_setting, img_size_setting))  
                training_data.append([resize_array, class_num])
                
            except Exception as e:  
                pass

    
create_training_data()           
 

    

100%|██████████| 202/202 [00:00<00:00, 1111.83it/s]


In [3]:

# Preprocessing

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
  

X = np.array(X).reshape(-1, img_size_setting, img_size_setting, 3) # reshaping list to array

# We dont need to normalize X between 0 and 1. I think VGG original model did not do it.
# If we do normalization here, like X= X/255.0, validation accuracy decreases.

print("Label of Images:= \n", y)

print("---------------------------------")
print("Shape of Images:= \n", X.shape) 
print("---------------------------------")
print("First Image:= \n ",  X[0])

Label of Images:= 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
# Hot hot encoding of y (labels)

Y = np_utils.to_categorical(y, 4)

# shuffling is required to avoid any element of bias/patterns in the split datasets before training the ML model.
X,Y = shuffle(X,Y, random_state=2)

In [5]:
# Train_test-split using sklearn

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)


#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary() 


# Changing classifier last layer as we have only 4 classes
# No. of parameters remains same

last_layer = model.get_layer('fc2').output
out = Dense(4, activation='softmax', name='output')(last_layer)
TL_vgg_model = Model(image_input, out)
TL_vgg_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
# Freezing weights of Vgg model till fc2 layer(second last layer)
# No. of trainable parameters decreases and no. of non-trainable parameters increases

for layer in TL_vgg_model.layers[:-1]:
	layer.trainable = False

TL_vgg_model.summary() # check summary 

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
# Model compliation and training with our dataset
# optimizer could be changed as per your choice

TL_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


hist = TL_vgg_model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))


# As no. of custom dataset is very small, TL_vgg_model learns quite well or we could say model is over fit

Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 5s 7ms/step - loss: 0.4324 - accuracy: 0.8607 - val_loss: 0.0818 - val_accuracy: 0.9630
Epoch 2/5
646/646 [==============================] - 2s 3ms/step - loss: 0.0606 - accuracy: 0.9814 - val_loss: 0.1328 - val_accuracy: 0.9568
Epoch 3/5
646/646 [==============================] - 2s 3ms/step - loss: 0.0131 - accuracy: 0.9985 - val_loss: 0.1307 - val_accuracy: 0.9568
Epoch 4/5
646/646 [==============================] - 2s 3ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0408 - val_accuracy: 0.9753
Epoch 5/5
646/646 [==============================] - 2s 3ms/step - loss: 0.0114 - accuracy: 0.9923 - val_loss: 0.0228 - val_accuracy: 0.9877


In [8]:
# Model performance on test samples from our dataset

(loss, accuracy) = TL_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

162/162 [==============================] - 1s 6ms/step
loss=0.0228, accuracy: 98.7654%
